Задания к [первой практике](https://colab.research.google.com/drive/1AKldHA9Ib8eQk4yn3yAi-Ffs4EtatPsh?usp=sharing).

### Задание 1



Сбор данных из ИСУ.

Выберите какой-либо факультет, соберите данные по обучающимся на нём группам бакалавров на всех курсах (ИСУ - Учебный процесс - Учебные группы и потоки). Сформируйте таблицу с колонками `группа`, `курс`, `номер_ису`, `фио`. Ответьте на вопросы:

1. Одинаковое ли количество групп от курса к курсу?

2. Есть ли в вашей выборке однофамильцы (для простоты учитывайте только полное совпадение фамилии)? Если да, выведите этих людей.

3. Есть ли в вашей выборке ребята без отчества? Если да, выведите этих людей.

4. Найдите курс, на котором меньше всего обучающихся. Выведите значение и номер курса. Если таких курсов несколько, выведите их все.

5. Для каждого курса найдите группу, в которой больше всего обучающихся. Выведите тройки (курс, группа, кол-во обучающихся). Если для какого-либо курса таких групп окажется несколько, выведите их все.

Старайтесь писать код так, чтобы при изменении наименования факультета ничего не ломалось.

In [ ]:
# Это необходимо, чтобы запустить код в Google Colab. Источник: https://nariyoo.com/python-how-to-run-selenium-in-google-colab/
%%capture
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium chromedriver-autoinstaller

In [ ]:
import sys
import pandas as pd
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller


sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--verbose')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--window-size=1920, 1200')
chrome_options.add_argument('--disable-dev-shm-usage')

chromedriver_autoinstaller.install()
driver = webdriver.Chrome(options=chrome_options)  # создание объекта драйвера

In [ ]:
# вход в ИСУ
driver.get("https://id.itmo.ru/login")

# подождать, пока подгрузится страница авторизации
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "username"))  # если случится TimeoutException - перезапустите ячейку
)

<selenium.webdriver.remote.webelement.WebElement (session="ef9091ac2861bd1b5b92a2f63db9d3c0", element="f.75C3F0DBEC3B193844E4A82CBAF7F7B2.d.22D4795D75AC2C4FCFDAD39841368596.e.11")>

In [ ]:
# информация для входа в ИСУ
username = '367179'
password = 'jejWun-gecda0-gohvug' # пароль!

# ввод данных для входа
username_field = driver.find_element(By.NAME, "username")
password_field = driver.find_element(By.NAME, "password")

username_field.send_keys(username)
password_field.send_keys(password)
password_field.send_keys(Keys.RETURN)

In [ ]:
target_faculty_name = 'факультет биотехнологий' # вставьте факультет
driver.get('https://isu.ifmo.ru')

In [ ]:
# переход в "учебные группы и потоки"
link_to_target_page = driver.find_element('xpath', "//a[contains(., 'Учебные группы и потоки')]")
target_url = link_to_target_page.get_attribute('href')
driver.get(target_url)

In [ ]:
group_urls = []
group_objects = driver.find_elements(By.XPATH, f"//h4[text() = 'Магистратура']/preceding-sibling::a[preceding-sibling::div[@class='note padding-sm fac-break'][1]/descendant::a[text()='{target_faculty_name}'] and following-sibling::div[@class='note padding-sm fac-break'][1]]")
for group_object in group_objects:
      group_urls.append(group_object.get_attribute("href"))
print(group_urls)

['https://isu.ifmo.ru/pls/apex/f?p=2143:GR:131840833430317::NO::GR_GR,GR_DATE,GR_TYPE:t3130,05.04.2024,group', 'https://isu.ifmo.ru/pls/apex/f?p=2143:GR:131840833430317::NO::GR_GR,GR_DATE,GR_TYPE:t3131,05.04.2024,group', 'https://isu.ifmo.ru/pls/apex/f?p=2143:GR:131840833430317::NO::GR_GR,GR_DATE,GR_TYPE:t3132,05.04.2024,group', 'https://isu.ifmo.ru/pls/apex/f?p=2143:GR:131840833430317::NO::GR_GR,GR_DATE,GR_TYPE:t3230,05.04.2024,group', 'https://isu.ifmo.ru/pls/apex/f?p=2143:GR:131840833430317::NO::GR_GR,GR_DATE,GR_TYPE:t3231,05.04.2024,group', 'https://isu.ifmo.ru/pls/apex/f?p=2143:GR:131840833430317::NO::GR_GR,GR_DATE,GR_TYPE:t3232,05.04.2024,group', 'https://isu.ifmo.ru/pls/apex/f?p=2143:GR:131840833430317::NO::GR_GR,GR_DATE,GR_TYPE:t33301,05.04.2024,group', 'https://isu.ifmo.ru/pls/apex/f?p=2143:GR:131840833430317::NO::GR_GR,GR_DATE,GR_TYPE:t33304,05.04.2024,group', 'https://isu.ifmo.ru/pls/apex/f?p=2143:GR:131840833430317::NO::GR_GR,GR_DATE,GR_TYPE:t33305,05.04.2024,group', 'https

In [ ]:
data = {"группа": [], "курс": [], "номер_ису": [], "фио": []} #"фамилия": [], "имя": [], "отчество": []} # добавим последние столбцы для дальнейшей работы
for i, groups in enumerate(group_urls):
    match = re.search(r'TYPE:[a-zA-Z]\d{4,5}', groups) # находим номер группы в url адресе
    if match:
        group_number = match.group(0)[5:]
        course = match.group(0)[7]

    driver.get(groups) # переходим на группу

    id = driver.find_elements(By.XPATH, '//td[@headers="ИД"]')
    isu = [n.text for n in id] # получаем ИСУ

    people = driver.find_elements('xpath', '//td[@headers="ФИО"]/a')
    people_names = []
    for person in people: # получаем имена
        student = person.get_attribute('text')
        people_names.append(student)
        fio = student.split(' ')
        name = fio[0]
        surname = fio[1]
        if len(fio) == 2:
            patronymic = ""
        else:
            patronymic = " ".join(fio[2:])

    data['группа'].extend([group_number for _ in range(len(people_names))])
    data['курс'].extend([course for _ in range(len(people_names))])
    data['номер_ису'].extend(isu)
    data['фио'].extend(people_names)

In [ ]:
df = pd.DataFrame(data)
df.sample(20)

,группа,курс,номер_ису,фио
216,t34302,4,309615,Кислицына Ксения Вадимовна
65,t3132,1,409275,Орехова Александра Дмитриевна
17,t3130,1,409231,Неустроев Александр Константинович
53,t3132,1,439069,Баглай Алисия Сергеевна
248,t34306,4,312568,Куранова Марина Алексеевна
163,t33304,3,336576,Зверева Вероника Андреевна
91,t3230,2,368682,Попышева Татьяна Дмитриевна
140,t3232,2,373448,Погодаева Софья Сергеевна
121,t3231,2,368969,Филозоп Владислав Сергеевич
166,t33304,3,352411,Павлова Елизавета Дмитриевна


1. Одинаковое ли количество групп от курса к курсу?

In [ ]:
group_amounts = df.groupby("курс")["группа"].nunique()
for course, count in group_amounts.items():
    print(f"Курс {course}: {count} групп")

all_equal = all(count == group_amounts[0] for count in group_amounts)

if all_equal:
    print("Количество групп одинаковое во всех курсах.")
else:
    print("Количество групп не одинаковое во всех курсах.")

Курс 1: 3 групп
Курс 2: 3 групп
Курс 3: 5 групп
Курс 4: 6 групп
Количество групп не одинаковое во всех курсах.


2. Есть ли в вашей выборке однофамильцы (для простоты учитывайте только полное совпадение фамилии)? Если да, выведите этих людей.

In [ ]:
# Вариант с методом groupby

grouped = df.groupby(df['фио'].str.split().str[0])

# Отфильтровываем группы, в которых больше одного человека
duplicates = grouped.filter(lambda x: len(x) > 1)

if duplicates.empty:
    print("Таких людей нет")
else:
    print("Найденные однофамильцы:")
    display(duplicates)

Найденные однофамильцы:


,группа,курс,номер_ису,фио
10,t3130,1,408939,Кузнецова Евгения Антоновна
11,t3130,1,408940,Кузнецова Мария Владиславовна
13,t3130,1,409058,Максимова Богдана Владимировна
33,t3131,1,408680,Иванова Анастасия Сергеевна
35,t3131,1,439869,Кузнецова Анастасия Сергеевна
43,t3131,1,409382,Попова Варвара Владимировна
44,t3131,1,368681,Попова Оксана Валерьевна
59,t3132,1,408685,Иванова Мария Николаевна
66,t3132,1,409384,Попова Елизавета Александровна
95,t3230,2,368816,Смирнова Арина Евгеньевна


3. Есть ли в вашей выборке ребята без отчества? Если да, выведите этих людей.

In [ ]:
# Функция для проверки наличия отчества
def has_patronymic(name):
    return len(name.split()) > 2

# Применяем функцию к столбцу 'фио' и проверяем дубликаты
without_patronymic = df[df['фио'].apply(lambda x: not has_patronymic(x))].drop_duplicates()

if without_patronymic.empty:
    print("Людей без отчества нет")
else:
    print("Люди без отчества:")
    display(without_patronymic)

Люди без отчества:


,группа,курс,номер_ису,фио
39,t3131,1,409196,Мусил Петр
50,t3131,1,409819,Цоллер Алина
147,t3232,2,373452,Уметалиева Айдай
155,t33301,3,336626,Кильдеева Дарья
196,t33321,3,339096,Валеева Валерия
205,t33321,3,339798,Репин Алексей


4. Найдите курс, на котором меньше всего обучающихся. Выведите значение и номер курса. Если таких курсов несколько, выведите их все.

In [ ]:
course_counts = df['курс'].value_counts()
min_courses = course_counts[course_counts == course_counts.min()].index.tolist()
print("Курсы с минимальным количеством студентов:")
for course in min_courses:
    print(f"Курс: {course}, Количество студентов: {course_counts[course]}")

Курсы с минимальным количеством студентов:
Курс: 4, Количество студентов: 52


5. Для каждого курса найдите группу, в которой больше всего обучающихся. Выведите тройки (курс, группа, кол-во обучающихся). Если для какого-либо курса таких групп окажется несколько, выведите их все.

In [ ]:
group_counts = df.groupby(['курс', 'группа']).size()

# Находим индексы (курс и группу), где количество студентов максимально для каждого курса
max_groups = group_counts.groupby(level='курс').apply(lambda x: x[x == x.max()].index.tolist())

print("Группы, в которых больше всего студентов:")
for course, groups in max_groups.items():
    for group in groups:
        group_number = group[1]  # Номер группы из кортежа (курс, группа)
        print(f"Курс: {course} Группа: {group_number} Количество студентов: {group_counts.loc[(course, group_number)]}")

Группы, в которых больше всего студентов:
Курс: 1 Группа: t3130 Количество студентов: 26
Курс: 2 Группа: t3230 Количество студентов: 27
Курс: 3 Группа: t33305 Количество студентов: 20
Курс: 4 Группа: t34306 Количество студентов: 16


### Задание 2

Одни из самых ужасных с точки зрения формата представления данные - это данные на сайтах государственных организаций (поликлиники, школы и т.п.). Потренируемся работать с подобными сайтами. Соберем данные за муниципальный этап Всероссийской олимпиады школьников [2022/2023 года по Томской области](http://rcro.tomsk.ru/munitsipal-ny-j-e-tap-vsosh-2022-23-arhiv/).

БОльшая часть кода уже написана за вас, однако вам необходимо дополнить функционал по непосредственному извлечению pandas-датафрейма из файлов различного формата (`pdf`, `xlsx`, `docx`), а также посчитать статистические показатели по собранным данным.

Изучите данные на сайте - таблицы имеют схожую, хоть и слегка различающуюся, структуру. Приведем извлеченную из каждого файла таблицу к датафрейму с колонками `предмет`, `класс`, `фио`, `баллы`. Если что-либо отсутствует - заполним пустой строкой, если речь идёт не о баллах, иначе нулем. ФИО можно расматривать как склейку через пробел полей "Фамилия", "Имя" и "Отчество" в случае наличия последнего.

Обратите внимание, что вы можете менять любую часть кода, добавлять свои функции, а также произвести парсинг полностью вручную (хотя это нежелательно).

In [ ]:
!pip install python-docx xls2xlsx img2table[easyocr]

In [ ]:
from img2table.document import PDF
from img2table.ocr import EasyOCR
from xls2xlsx import XLS2XLSX
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
from docx import Document


easyocr = EasyOCR(lang=["ru"], kw={"gpu": False})  # можно использовать любой другой OCR-движок

In [ ]:
def get_file_extension(filename):
    ext = filename.split('.')[-1]
    if ext not in ['xls', 'xlsx', 'pdf', 'docx']:
        raise ValueError
    return ext


def download_file(url, destination):
    response = requests.get(url)
    if response.status_code == 200:
        with open(destination, 'wb') as f:
            f.write(response.content)
        print(f"File downloaded successfully to '{destination}'")
    else:
        print(f"Failed to download file from '{url}'")


def convert_xls_to_xlsx(filename):  # конвертация xlsx-файлов устаревшего образца
    x2x = XLS2XLSX(filename)
    x2x.to_xlsx(filename + "x")
    return filename


def extract_data(subject, grade, filename, ext, verbose=True):
    year = filename.split("_")[1].split(".")[0] # находим класс
    if ext == 'xlx':
        ext = 'xlsx'
        filename = convert_xls_to_xlsx(filename)

    if ext == 'xlsx':
        df = parse_xlsx(filename)
        df['баллы'] = df['баллы'].astype(str) # чтобы в конце не возникли NaN
    elif ext == 'pdf':
        df = parse_pdf(filename)
    elif ext == 'docx':
        df = parse_docx(filename)

    df['предмет'] = subject
    df['класс'] = year

    if verbose:
        print()
        display(df)
        print()
    return df[['предмет', 'класс', 'фио', 'баллы']]

def parse_xlsx(filename):
    df1 = pd.read_excel(filename, header=3)
    df1.drop(['№', 'Образовательная организация', 'Муниципалитет', 'Класс обучения'], axis=1, inplace=True)
    df1["фио"] = df1['Фамилия'] + ' ' + df1['Имя']
    df1.drop(['Фамилия', 'Имя'], axis=1, inplace=True)
    df1 = df1.rename(columns = {"Балл": "баллы"})
    return df1


def parse_docx(filename):
    document = Document(filename)
    table = document.tables[0]
    data = [[cell.text for cell in row.cells] for row in table.rows]
    df = pd.DataFrame(data)
    df = df.rename(columns=df.iloc[0])
    df = df.drop(df.index[0])
    fio_column = []
    for index, row in df.iterrows():
        if pd.isna(row['Отчество']):
            fio_column.append(f"{row['Фамилия']} {row['Имя']}")
        else:
            fio_column.append(f"{row['Фамилия']} {row['Имя']} {row['Отчество']}")
    columns_to_drop = list(range(0, 6)) + list(range(7, len(df.columns)))
    df.drop(df.columns[columns_to_drop], axis=1, inplace=True)
    df["фио"] = fio_column
    df = df.rename(columns = {"Результат": "баллы"})
    return df

def parse_pdf(filename):
    doc = PDF(filename,
              detect_rotation=True,
              pdf_text_extraction=True)
    extracted_tables = doc.extract_tables(ocr=easyocr,
                                          implicit_rows=False,
                                          borderless_tables=False,
                                          min_confidence=50)

    tables = []
    for page in extracted_tables.values():
        tables.extend(page) # все таблицы


    df_list = []

    for table in tables:
        df = table.df
        df_list.append(df)
    combined_df = pd.concat(df_list, ignore_index=True)

    data = []
    combined_df = combined_df.rename(columns=combined_df.iloc[0])
    combined_df = combined_df.drop(combined_df.index[0])
    for index, row in combined_df.iterrows():
            points = row.iloc[-2]
            if points == '' or points is None:
                points = '0'
            if pd.isna(row['Отчество']):
                name = f"{row['Фамилия']} {row['Имя']}"
            else:
                name = f"{row['Фамилия']} {row['Имя']} {row['Отчество']}"
            data.append({"баллы": points, "фио": name})
    #display(data)
    df1 = pd.DataFrame(data)
    return df1

Для простоты будем работать только со следующим списком предметов: экономика, английский язык, математика, биология, информатика и ИКТ. Следующий код менять не нужно.

In [ ]:
target_subjects = ['Экономика', 'Английский язык', 'Математика', 'Биология', 'Информатика и ИКТ']  # рассмотрим только эти предметы

all_dfs = []
response = requests.get('http://rcro.tomsk.ru/munitsipal-ny-j-e-tap-vsosh-2022-23-arhiv/')  # результаты муниципального этапа за 2022-2023 год
main_table = BeautifulSoup(response.text).find('table').findAll('table')[2]  # основная таблица (см. сайт)

for row in main_table.findAll('tr')[1:]:  # первая строка - шапка таблицы
    cells = row.findAll('td')

    subject = cells[0].text.strip()  # название предмета из таблицы
    if subject not in target_subjects:
        continue

    a_hrefs = cells[-1].findAll('a')
    for a_href in a_hrefs:  # ссылки на файлы
        grade = re.search(r'\d+', a_href.text).group(0)
        href = a_href.get('href')

        ext = get_file_extension(href)
        filename = f"{subject}_{grade}.{ext}"
        download_file(href, filename)  # скачивание файла

        extracted_df = extract_data(subject, grade, filename, ext, verbose=True)  # достать данные из файла
        all_dfs.append(extracted_df)

overall_df = pd.concat(all_dfs, ignore_index=True)
overall_df.loc[overall_df['баллы'] == '', 'баллы'] = 0  # фикс для NaN-ов в случае баллов
overall_df['баллы'] = overall_df['баллы'].str.replace(',', '.').astype(float)  # конвертация баллов в числовой формат

File downloaded successfully to 'Экономика_9.xlsx'



,баллы,фио,предмет,класс
0,84,Зюбин Владислав,Экономика,9
1,74,Кириллов Иван,Экономика,9
2,74,Телицин Артемий,Экономика,9
3,72,Шарабарина Елизавета,Экономика,9
4,67,Кочетов Дмитрий,Экономика,9
5,66,Колмыков Арсений,Экономика,9
6,66,Большанин Матвей,Экономика,9
7,65,Котлевский Дмитрий,Экономика,9
8,65,Правосуд Юлия,Экономика,9
9,64,Винюкова Виктория,Экономика,9



File downloaded successfully to 'Экономика_10.pdf'



,баллы,фио,предмет,класс
0,71,Соколов Владислав Андреевич,Экономика,10
1,67,Танасейчук Тимофей Владимирович,Экономика,10
2,64,Шашко Максим Дмитриевич,Экономика,10
3,63,Аникин Лев Дмитриевич,Экономика,10
4,61,Сухоруков Николай Максимович,Экономика,10
5,59,Горн Максим Викторович,Экономика,10
6,56,Фещенко Михаил Артёмович,Экономика,10
7,52,Куперт Людмила Александровна,Экономика,10
8,50,Колтунов Антон Витальевич,Экономика,10
9,50,Матвеева Полина Александровна,Экономика,10



File downloaded successfully to 'Экономика_11.pdf'



...

,баллы,фио,предмет,класс
0,381,Лисок Артем Александрович,Информатика и ИКТ,9
1,329,Лисок Никита Александрович,Информатика и ИКТ,9
2,303,Александров Илья Александрович,Информатика и ИКТ,9
3,255,Ивасенко Илья Данилович,Информатика и ИКТ,9
4,252,Полубоярцев Данил Дмитриевич,Информатика и ИКТ,9
5,228,Куликов Ярослав Глебович,Информатика и ИКТ,9
6,204,Гахов Артем Борисович,Информатика и ИКТ,9
7,200,Локотецкий Александр Анатольевич,Информатика и ИКТ,9
8,195,Чуйко Алексей Игоревич,Информатика и ИКТ,9
9,161,Ефимов Артем Игоревич,Информатика и ИКТ,9



File downloaded successfully to 'Информатика и ИКТ_10.pdf'



,баллы,фио,предмет,класс
0,388,Осипов Вячеслав Витальевич,Информатика и ИКТ,10
1,286,Атимасов Никита Игоревич,Информатика и ИКТ,10
2,274,Ильин Василий Игоревич,Информатика и ИКТ,10
3,255,Будаев Никита Олегович,Информатика и ИКТ,10
4,250,Соколов Владислав Андреевич,Информатика и ИКТ,10
5,235,Борисевич Иван Андреевич,Информатика и ИКТ,10
6,233,Михайлов Павел Алексеевич,Информатика и ИКТ,10
7,233,Жуков Сергей Владимирович,Информатика и ИКТ,10
8,233,Трофимов Юрий Алексеевич,Информатика и ИКТ,10
9,233,Малахов Кирилл Антонович,Информатика и ИКТ,10



File downloaded successfully to 'Информатика и ИКТ_11.pdf'



,баллы,фио,предмет,класс
0,500,Агафонов Артём Александрович,Информатика и ИКТ,11
1,500,Зюбин Константин Сергеевич,Информатика и ИКТ,11
2,461,Зиновьев Алексей Игоревич,Информатика и ИКТ,11
3,429,Разинков Иван Константинович,Информатика и ИКТ,11
4,429,Фокин Степан Дмитриевич,Информатика и ИКТ,11
5,357,Литвинов Никита Владимирович,Информатика и ИКТ,11
6,337,Русанова Маргарита Игоревна,Информатика и ИКТ,11
7,320,Богданова Олеся Александровна,Информатика и ИКТ,11
8,318,Горбатенко Александр Александрович,Информатика и ИКТ,11
9,318,Малютин Артём Антонович,Информатика и ИКТ,11


Должно получиться что-то похожее:

In [ ]:
overall_df

,предмет,класс,фио,баллы
0,Экономика,9,Зюбин Владислав,84.00
1,Экономика,9,Кириллов Иван,74.00
2,Экономика,9,Телицин Артемий,74.00
3,Экономика,9,Шарабарина Елизавета,72.00
4,Экономика,9,Кочетов Дмитрий,67.00
5,Экономика,9,Колмыков Арсений,66.00
6,Экономика,9,Большанин Матвей,66.00
7,Экономика,9,Котлевский Дмитрий,65.00
8,Экономика,9,Правосуд Юлия,65.00
9,Экономика,9,Винюкова Виктория,64.00


Постройте сводную таблицу - по каждому предмету для каждого класса посчитайте минимальный балл, средний балл, медианный балл, а также максимальный балл.

Посмотрите на результаты по информатике. О чём могут свидетельствовать подобные цифры?

In [ ]:
import numpy as np

overall_df['баллы'] = overall_df['баллы'].apply(lambda val: float(str(val).replace(',', '.')))  # конвертация баллов в числовой формат

def summary_table(subject):   # создаем сводную таблицу
    pivot_table = pd.pivot_table(overall_df, index=['класс', 'предмет'],
                                 values='баллы',
                                 aggfunc={'баллы': [np.min, np.mean, np.median, np.max]})
    subject_results = pivot_table.loc[(slice(None), subject), :]
    return subject_results

subjects = ['Экономика', 'Английский язык', 'Математика', 'Биология', 'Информатика и ИКТ']

summary_df = pd.DataFrame()

for subject in subjects:
    subject_summary = summary_table(subject)
    summary_df = pd.concat([summary_df, subject_summary])

display(summary_df)


,,max,mean,median,min
класс,предмет,,,,
10,Экономика,71.0,26.863095,23.0,1.0
11,Экономика,69.0,32.058586,32.0,4.0
9,Экономика,84.0,28.518072,23.0,0.0
10,Английский язык,54.0,24.628205,24.0,0.0
11,Английский язык,46.0,26.286111,28.0,0.0
9,Английский язык,45.0,21.319643,23.0,1.0
10,Математика,26.0,4.201754,1.0,0.0
11,Математика,28.0,6.160194,7.0,0.0
8,Математика,28.0,2.903670,2.0,0.0
